In [1]:
import ngsolve as ngs
from do import solve
from ngsolve.webgui import Draw
import matplotlib.pyplot as plt
from copy import deepcopy

levels = 4
deg = 1

######################################################
from bhdata import BHCurves
fun_dw  = BHCurves(4)
fun_w   = fun_dw.Integrate()
fun_ddw = fun_dw.Differentiate()

mu0 = 1.256636e-6
nu0 = 1/mu0

linear = "coil_plus|coil_minus"
nonlinear = "stator"

strom = 1e5
amps = 0.025**2*3.15*strom
print(f'Applying a current of {amps:.2f} Amps')
######################################################

from createGeom import makeGeo
geoOCC = makeGeo()

BH data from TEAM 13 problem
Applying a current of 196.88 Amps


In [2]:
meshes = []; Bs = []; Bps = []; its = [];

for i in range(levels):
    ngmesh = geoOCC.GenerateMesh()
    mesh = ngs.Mesh(ngmesh)

    for j in range(i):
        mesh.Refine()
    mesh.Curve(3)

    with ngs.TaskManager(): J = mesh.MaterialCF({'coil_plus': (0,0,strom), 'coil_minus': (0,0,-strom), 'stator': (0,0,0)}, default = (0,0,0))
    HCurl = ngs.HCurl(mesh, order = deg, nograds = True, dirichlet = 'outer_face|coil_plus_face|coil_minus_face|stator_face', autoupdate = True)
    V3L2 = ngs.L2(mesh, dim = 3, order = deg-1, autoupdate = True)
    A0 = ngs.GridFunction(HCurl)
    A0, it = solve(HCurl, A0, mesh,deg, J, fun_w, fun_dw, fun_ddw)
    its.append(it)

    B = ngs.GridFunction(V3L2, nested = True)
    with ngs.TaskManager(): B.Set(ngs.curl(A0))
    Bs.append(B)

    if i>0:
        V3L2 = ngs.L2(mesh, dim = 3, order = deg-1)
        Bp = ngs.GridFunction(V3L2)
        with ngs.TaskManager(): Bp.Set(Bs[i-1])
        Bps.append(Bp)
        with ngs.TaskManager(): error = ngs.Integrate((B-Bp)**2, mesh)*(1/2)
        print(error)
        

    # ######################################################
    # # Refine mesh, apply prolongation
    # ######################################################

    # ngmesh2 = geoOCC.GenerateMesh()
    # mesh2 = ngs.Mesh(ngmesh2)

    # for j in range(i+1):
    #     mesh2.Refine()
    # mesh2.Curve()

    # V3L2 = ngs.L2(mesh2, dim=3, order = deg-1)
    # Bp = ngs.GridFunction(V3L2)
    # Bp.Set(B)
    # Bps.append(B)


print(its)


Using 3D mesh with ne= 1057 elements and nv= 307 points and  1580 DOFs.
 
Using 3D mesh with ne= 8456 elements and nv= 1887 points and  12790 DOFs.
 
5.328011346541991e-06
Using 3D mesh with ne= 67648 elements and nv= 13097 points and  97006 DOFs.
 
2.2467046595046153e-06
Using 3D mesh with ne= 541184 elements and nv= 97313 points and  749390 DOFs.
 
6.508678547245232e-07
[8, 8, 8, 8]


In [3]:
import numpy as np
a = np.array([5.332232056354972e-06, 2.246711173326673e-06, 6.508678547245244e-07])
np.log2(a[:-1]/a[1:])

array([1.2469249, 1.7873781])

In [4]:
######################################################
# Compute first solution
######################################################

ngmesh = geoOCC.GenerateMesh()
mesh = ngs.Mesh(ngmesh)
# mesh.Refine()
mesh.Curve(3)

levels = 2
deg = 1

J = mesh.MaterialCF({'coil_plus': (0,0,strom), 'coil_minus': (0,0,-strom), 'stator': (0,0,0)}, default = (0,0,0))

HCurl = ngs.HCurl(mesh, order = deg, nograds = True, dirichlet = 'outer_face|coil_plus_face|coil_minus_face|stator_face', autoupdate = True)
V3L2 = ngs.L2(mesh, dim=3, order = deg-1, autoupdate = True)

A0 = ngs.GridFunction(HCurl)
A0, it0 = solve(HCurl,A0,mesh,deg,J,fun_w,fun_dw,fun_ddw)

B0 = ngs.GridFunction(V3L2, nested = True)
with ngs.TaskManager(): B0.Set(ngs.curl(A0))

clipping = { "function" : True, "z" : -0.4}
with ngs.TaskManager(): Draw(B0, mesh, clipping = clipping)

######################################################
# Refine mesh, apply prolongation
######################################################

ngmesh2 = geoOCC.GenerateMesh()
mesh2 = ngs.Mesh(ngmesh2)

mesh2.Refine()
# mesh2.Refine()
mesh2.Curve(3)

V3L2 = ngs.L2(mesh2, dim=3, order = deg-1)
B01 = ngs.GridFunction(V3L2)
B01.Set(B0)

clipping = { "function" : True, "z" : -0.4}
with ngs.TaskManager(): Draw(B01, mesh2, clipping = clipping)

######################################################
# Compute second solution
######################################################

HCurl = ngs.HCurl(mesh2, order = deg, nograds = True, dirichlet = 'outer_face|coil_plus_face|coil_minus_face|stator_face', autoupdate = True)
V3L2 = ngs.L2(mesh2, dim=3, order = deg-1)

A1 = ngs.GridFunction(HCurl)
A1, it1 = solve(HCurl,A1,mesh2,deg,J,fun_w,fun_dw,fun_ddw)

B1 = ngs.GridFunction(V3L2, nested = True)
with ngs.TaskManager(): B1.Set(ngs.curl(A1))

clipping = { "function" : True, "z" : -0.4}
with ngs.TaskManager(): Draw(B1, mesh2, clipping = clipping)


######################################################
# Compute error
######################################################

clipping = { "function" : True, "z" : -0.4}
with ngs.TaskManager(): Draw(B01-B1, mesh2, clipping = clipping)

error = ngs.Integrate((B01-B1)**2, mesh)*(1/2)
error


Using 3D mesh with ne= 1057 elements and nv= 307 points and  1580 DOFs.
 


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

Using 3D mesh with ne= 8456 elements and nv= 1887 points and  12790 DOFs.
 


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

4.777482607243841e-06

In [5]:
ngmesh2 = geoOCC.GenerateMesh()
mesh2 = ngs.Mesh(ngmesh2)

mesh2.Refine()
mesh2.Refine()
mesh2.Curve(3)

V3L2 = ngs.L2(mesh2, dim=3, order = deg-1)
B01 = ngs.GridFunction(V3L2)
B01.Set(B0)

clipping = { "function" : True, "z" : -0.4}
with ngs.TaskManager(): Draw(B01, mesh2, clipping = clipping)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

In [6]:
HCurl = ngs.HCurl(mesh2, order = deg, nograds = True, dirichlet = 'outer_face|coil_plus_face|coil_minus_face|stator_face', autoupdate = True)
A1 = ngs.GridFunction(HCurl)
A1, it1 = solve(HCurl,A1,mesh2,deg,J,fun_w,fun_dw,fun_ddw)

V3L2 = ngs.L2(mesh2, dim=3, order = deg-1)
B1 = ngs.GridFunction(V3L2, nested = True)
with ngs.TaskManager(): B1.Set(ngs.curl(A1))

clipping = { "function" : True, "z" : -0.4}
with ngs.TaskManager(): Draw(B1, mesh2, clipping = clipping)

Using 3D mesh with ne= 67648 elements and nv= 13097 points and  97006 DOFs.
 


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

In [7]:
error = ngs.Integrate((B01-B1)**2, mesh)*(1/2)
error

7.30165737153249e-06

In [10]:

np.log2(0.00017818059050590164/3.2399858246322277e-05)

2.4592807841074844